In [7]:
import os
import zipfile as zip
import pandas as pd
import csv
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from fuzzywuzzy import fuzz
import ast
import datetime
import dateparser as dp
from sklearn.metrics.pairwise import cosine_similarity
from concentrationMetrics import Index

In [8]:
os.chdir("E:\data")

In [27]:
pd.read_csv("GCPD_granular_data.txt", nrows=3)

,nr_pt,gvkey,conml,assg_name,date_issue,date_filing
0,9642295,202332,Fuji Corp,"Fuji Machine Mfg. Co., Ltd.",02may2017,11nov2013
1,9642291,213207,Inventec Corp,Inventec Corporation,02may2017,17may2016
2,9642289,132740,Infineon Technologies AG,Infineon Technologies Austria Ag,02may2017,19sep2013


In [28]:
pd.read_csv('top10pat.csv', nrows=3)

,Unnamed: 0,nr_pt,gvkey,conml,assg_name,date_issue,date_filing
0,312,9641815,6266,Johnson & Johnson,"DePuy Synthes Products, Inc.",2017-05-02,2014-03-14
1,1502,9639917,101310,Novartis AG,Novartis Ag,2017-05-02,2015-10-07
2,2180,9638936,6266,Johnson & Johnson,"Johnson & Johnson Vision Care, Inc.",2017-05-02,2014-08-20


In [33]:
pd.read_csv('patent_inventor.tsv', delimiter='\t', nrows=3)

,patent_id,inventor_id,location_id
0,6584128,6584128-1,NaN
1,4789863,4789863-1,NaN
2,6795487,6795487-2,NaN


In [34]:
pd.read_csv('sampleframe.csv', nrows=3)

,Unnamed: 0,patent_id,gvkey,conml,assg_name,date_issue,date_filing,inventor_id,location_id
0,0,9225375,6266,Johnson & Johnson,"Johnson & Johnson Vision Care, Inc.",2015-12-29,2013-09-23,8526113-6,bb1b1d58-4255-4421-92a9-4a781a2982b2
1,1,9225375,6266,Johnson & Johnson,"Johnson & Johnson Vision Care, Inc.",2015-12-29,2013-09-23,9014639-2,46b9b0dc-b7bd-4a34-98d3-66f4a6730ba4
2,2,9225375,6266,Johnson & Johnson,"Johnson & Johnson Vision Care, Inc.",2015-12-29,2013-09-23,7931832-1,28eb0edc-c70b-4111-9571-0a7ea7bd8f03


In [35]:
pd.read_csv('top10inventors.csv', nrows=3)

,Unnamed: 0,inventor_id,patent_id,class
0,0,8526113-6,9244203,G02B
1,1,8526113-6,9244203,G02C
2,7,8526113-6,9225375,A61B


In [37]:
pd.read_csv('ipc3.csv', nrows=3)

,Unnamed: 0,patent_id,class,group,uuid,main_group,subgroup,cms,cm
0,0,6864832,G01S,013/42,00005z3qh82fwpo5r1oupwpr3,13,42,G01S+13+42,G01S+13
1,1,9954111,H01L,027/1156,0000662nssr53hdo3lp92sz26,27,1156,H01L+27+1156,H01L+27
2,2,10048897,G06F,012/891,00008u9j3g8oivqtuc1dqayb1,12,891,G06F+12+891,G06F+12


In [38]:
pd.read_csv('uniqueinventors.csv', nrows=3)

,Unnamed: 0,inventor_id,patent_id,class_x,A00N,A01B,A01C,A01D,A01F,A01G,...,H06F,H07M,H09B,H11L,H22K,H61B,H61F,H61K,H61M,class_y
0,0,8526113-6,9244203,G02B,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
1,1,8526113-6,9244203,G02C,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
2,2,8526113-6,9225375,A61B,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4


In [39]:
pd.read_csv('sampleframe2.csv', nrows=3)

,Unnamed: 0,patent_id,gvkey,conml,assg_name,date_issue,date_filing,inventor_id,location_id,A00N,...,H05K,H06F,H07M,H09B,H11L,H22K,H61B,H61F,H61K,H61M
0,0,9225375,6266,Johnson & Johnson,"Johnson & Johnson Vision Care, Inc.",2015-12-29,2013-09-23,8526113-6,bb1b1d58-4255-4421-92a9-4a781a2982b2,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,9225375,6266,Johnson & Johnson,"Johnson & Johnson Vision Care, Inc.",2015-12-29,2013-09-23,9014639-2,46b9b0dc-b7bd-4a34-98d3-66f4a6730ba4,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,9225375,6266,Johnson & Johnson,"Johnson & Johnson Vision Care, Inc.",2015-12-29,2013-09-23,7931832-1,28eb0edc-c70b-4111-9571-0a7ea7bd8f03,0.0,...,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [40]:
pd.read_csv('vectors.csv', nrows=3)

,inventor_id,A00N,A01B,A01C,A01D,A01F,A01G,A01H,A01J,A01K,...,H05K,H06F,H07M,H09B,H11L,H22K,H61B,H61F,H61K,H61M
0,3931106-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3931229-2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3931268-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [41]:
pd.read_csv('top10portfolios.csv', nrows=3)

,inventor_id,patent_id,class
0,3931106-1,28,9
1,3931229-2,10,2
2,3931268-1,29,8
